# Cousera Capstone Project
- Codeset to accomplish the Cousera Capstone Project
## Methods
- To be updated as accomplished
## Outputs
-To be updated as accomplished

In [1]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course')

Hello Capstone Project Course
